In [1]:
import pandas as pd
import os
import numpy as np
import warnings
from sklearn.preprocessing import MinMaxScaler, StandardScaler

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
from statsmodels.stats.multitest import multipletests

In [2]:
seg_folder = 'outputs/metrics_ondri_cerebra/'
metric_df = []
seg_files = os.listdir(seg_folder)
for file in seg_files:
    metric_df.append(pd.read_csv(seg_folder + file))

metric_df = pd.concat(metric_df)
metric_df['SUBJECT'] = [x.split('.')[:-1][0] for x in seg_files]
# if 'TBV-voxel' in metric_df.columns:
#     metric_df = metric_df.drop(columns=['TBV-voxel'])

metric_df = metric_df.dropna(axis=1, how='all')


In [3]:
nvox_cols = metric_df.columns[metric_df.columns.str.contains('nvox')]


In [4]:
delete_cols = metric_df.columns[metric_df.columns.str.contains('MII')  | metric_df.columns.str.contains('mean')]

metric_df = metric_df.drop(delete_cols, axis=1)

In [5]:
clinical_df = pd.read_csv('data/summary/ONDRI_summary.csv')

In [6]:


for cohort in ['HC', 'ADMCI', 'PD', 'FTD', 'ALS']:
    print(cohort)
    # fs_t1_vols = pd.read_csv('outputs/fs_t1_volumes.csv')

    # nvox_cols = fs_t1_vols.columns[~fs_t1_vols.columns.str.contains('SUBJECT')]
    # fs_t1_vols[nvox_cols] = fs_t1_vols[nvox_cols].to_numpy()/np.expand_dims(fs_t1_vols['total_intracranial_volume'].to_numpy(), axis=-1)
    # fs_t1_vols = fs_t1_vols.rename(columns={x: x + '_nvox' for x in nvox_cols})
    merged = pd.merge(clinical_df[['SUBJECT', 'COHORT']], metric_df, on='SUBJECT')
    merged.columns = [x.replace('Left', 'Tempxxx') for x in merged.columns]
    merged.columns = [x.replace('Right', 'Left') for x in merged.columns]
    merged.columns = [x.replace('Tempxxx', 'Right') for x in merged.columns]

    # merged = pd.merge(merged, fs_t1_vols, on='SUBJECT', how='inner')
    # merged = merged.drop(columns=['total_intracranial_volume_nvox'])
    merged = merged[merged['COHORT'] == cohort]
    merged = merged.dropna()
    print(len(merged))

    scaler = StandardScaler()
    x = scaler.fit_transform(merged[merged.columns[merged.columns.str.contains('nvox')]])


    scaler = StandardScaler()
    y = scaler.fit_transform(merged[merged.columns[merged.columns.str.contains('MAD')]])

    from scipy.stats import pearsonr
    cols = merged[merged.columns[merged.columns.str.contains('nvox')]].columns

    rs = []
    for i, col in enumerate(cols):
        rs.append(pearsonr(x[:, i], y[:, i]))

    df = pd.DataFrame()
    df['region'] = cols
    df['r'] = [r.statistic for r in rs]
    test = multipletests([x.pvalue for x in rs])
    df['p'] = test[1]

    pd.set_option('display.precision', 2)
    significant = df[df['p'] < 0.05]
    significant['r'] = np.round(significant['r'], 2)


    significant['p'] = [float('%.3g' % x) for x in significant['p']]
    significant.sort_values('r', ascending=False, inplace=True)
    print(significant)
    significant.to_csv('outputs/MAD_size_correlation_atlas_%s.csv' % cohort, index=False)

HC
17
Empty DataFrame
Columns: [region, r, p]
Index: []
ADMCI
125
                                    region     r         p
14                   nvox_Left Paracentral -0.36  3.28e-03
100       nvox_Right Vermal lobules VI-VII -0.37  2.02e-03
49         nvox_Left Vermal lobules VI-VII -0.37  1.66e-03
80          nvox_Right Transverse Temporal -0.37  1.92e-03
77            nvox_Right Superior Parietal -0.37  2.26e-03
73                    nvox_Right Precuneus -0.37  1.49e-03
91                      nvox_Right Caudate -0.38  1.43e-03
65                  nvox_Right Paracentral -0.38  9.58e-04
36    nvox_Left Inferior Lateral Ventricle -0.40  4.47e-04
84             nvox_Right Fourth Ventricle -0.42  1.28e-04
40                       nvox_Left Caudate -0.43  4.17e-05
87   nvox_Right Inferior Lateral Ventricle -0.43  5.25e-05
10                       nvox_Left Lingual -0.43  5.85e-05
33              nvox_Left Fourth Ventricle -0.44  2.64e-05
29           nvox_Left Transverse Temporal -0.44 